First exploration of a new strategy: Inside Bar Momentum.  Simply:  if high-low of bar 2 within high-low of bar 1: signal...

In [40]:
import pandas as pd
import plotly.graph_objects as go
import utils

We know we are going to look for data on USD_JPY_H4, but we haven't yet tested collect_his_data with this pairing, so we need to run that first.  Might as well refresh while we're in the area...

But, this takes a long time to run - so comment out after doing it once.

In [41]:
# import os
# %run ./collect_his_data.py


In [42]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_csv(utils.get_hist_data_filename(pair, granularity))


In [43]:
non_nums = ['ticker', 'time', 'volume']
num_cols = [x for x in df_raw.columns if x not in non_nums]
df_raw[num_cols] = df_raw[num_cols].apply(pd.to_numeric)
df_raw.head()

,Unnamed: 0,ticker,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c
0,0,USD_JPY,2020-01-01T22:00:00.000000000Z,920,108.601,108.733,108.586,108.667,108.651,108.761,108.607,108.673,108.701,108.808,108.616,108.679
1,1,USD_JPY,2020-01-02T02:00:00.000000000Z,579,108.664,108.751,108.652,108.736,108.671,108.758,108.658,108.742,108.678,108.764,108.664,108.747
2,2,USD_JPY,2020-01-02T06:00:00.000000000Z,1418,108.738,108.836,108.688,108.833,108.744,108.842,108.696,108.839,108.750,108.848,108.703,108.845
3,3,USD_JPY,2020-01-02T10:00:00.000000000Z,1134,108.830,108.859,108.652,108.688,108.836,108.866,108.658,108.694,108.842,108.872,108.665,108.700
4,4,USD_JPY,2020-01-02T14:00:00.000000000Z,3273,108.691,108.742,108.206,108.523,108.697,108.748,108.212,108.529,108.703,108.754,108.219,108.535


In [44]:
df_raw.columns

Index(['Unnamed: 0', 'ticker', 'time', 'volume', 'bid_o', 'bid_h', 'bid_l',
       'bid_c', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_o', 'ask_h', 'ask_l',
       'ask_c'],
      dtype='object')

In [45]:
df = df_raw[['ticker', 'time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c']].copy()

In [46]:
def direction(row):
    """ Return price direction based on close vs open price i.e. if close higher, upwards (1)..."""
    if row.mid_c > row.mid_o:
        return 1
    return -1

In [47]:
def get_signal(row):
    """  Identify encapsulation in previous candle and return direction.  Pass back zero if no encapsulation.  
    1 = Buy, -1=Sell:  Buy if encapsulated in a previously upward candle...      """
    if row.mid_h_prev > row.mid_h and row.mid_l_prev > row.mid_l:
        return row.DIRECTION_prev
    return 0

## Additional column capture/derivation.

* The range of a candle is the difference between the high and low price.
* To track whether the high and low and enclosed in previous candle, we need high and low values from previous.



In [48]:
#TODO:  Create a new dataframe/sheet to capture Data Dictionary and some metadata...

df = df_raw[['ticker', 'time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_c']].copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['RANGE_PREV'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)
df.head()


,ticker,time,mid_o,mid_h,mid_l,mid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_PREV,DIRECTION,DIRECTION_prev,SIGNAL
0,USD_JPY,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.747,0.100,108.761,108.607,0.154,1,1,0
1,USD_JPY,2020-01-02T06:00:00.000000000Z,108.744,108.842,108.696,108.839,108.845,0.146,108.758,108.658,0.100,1,1,0
2,USD_JPY,2020-01-02T10:00:00.000000000Z,108.836,108.866,108.658,108.694,108.700,0.208,108.842,108.696,0.146,-1,1,0
3,USD_JPY,2020-01-02T14:00:00.000000000Z,108.697,108.748,108.212,108.529,108.535,0.536,108.866,108.658,0.208,-1,-1,-1
4,USD_JPY,2020-01-02T18:00:00.000000000Z,108.526,108.572,108.494,108.561,108.611,0.078,108.748,108.212,0.536,1,-1,0


Capture how many rows in our dataframe, and count how many buy/sell signals we're getting.

In [49]:
print(df.shape[0])
df.groupby(by="SIGNAL").count()

4674


,ticker,time,mid_o,mid_h,mid_l,mid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_PREV,DIRECTION,DIRECTION_prev
SIGNAL,,,,,,,,,,,,,
-1,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112,1112
0,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161,3161
1,401,401,401,401,401,401,401,401,401,401,401,401,401
